# PySpark Logistic Regression

Our full dataset file has around 9 million samples. When trying to run feature_generator.

Machine:
* 2018 Mac Mini - 6 core

Docker Configuration:
* 9 CPUs
* 24 GB Ram
* 3 GB swap


# References:

* https://spark.apache.org/docs/2.2.0/mllib-feature-extraction.html
* https://towardsdatascience.com/countvectorizer-hashingtf-e66f169e2d4e
* https://medium.com/@dhiraj.p.rai/logistic-regression-in-spark-ml-8a95b5f5434c
* packaging spark job - https://developerzen.com/best-practices-writing-production-grade-pyspark-jobs-cb688ac4d20f



In [1]:
import sys
sys.path.append("../..")

import pyspark
from pyspark import SparkContext, SparkFiles
from pyspark.sql import SparkSession, DataFrameReader, SQLContext
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import NGram, CountVectorizer, VectorAssembler, Tokenizer, IDF
from pyspark.ml import Pipeline
from pyspark.sql.functions import when, lit, col
from pyspark.sql.types import NumericType


from sklearn.utils.class_weight import compute_class_weight

import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from pprint import pprint
import logging

import util.pyspark_util as pyu
import util.model_wrapper as mw



log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)
sns.set()
%matplotlib inline

# allow DEBUG to be set by command line
DEBUG = bool(os.environ.get("IPYNB_DEBUG", False))

N_CLASSES = 5
FEATURE_COLUMN = "review_body"


if DEBUG:
    MIN_DF = 1
    DF_PERCENTAGE = 0.001
    DATA_FILE = "/home/jupyter/dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-test1k-preprocessed.csv"
    TEST_STR="-test"
else:
    DF_PERCENTAGE = 0.001
    DATA_FILE = "/home/jupyter/dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-100k-preprocessed.csv"
    TEST_STR=""

REPORT_FILE = f"../../reports/201911-pyspark-report{TEST_STR}.csv"


spark = SparkSession.builder \
            .master("local[*]") \
            .appName("Pyspark Wrapper Test (local)") \
            .getOrCreate()




In [2]:



class Timer(object):
    
    def __init__(self, description: str):
        self.start_time = datetime.now()
        self.description = description
        
    def stop(self):
        self.end_time = datetime.now()
        self.print_duration_min()
        
    def print_duration_min(self):
        self.duration = int((self.end_time - self.start_time).total_seconds() / 60)
        print(f"{self.description} duration: {self.duration} minutes")
        
    def get_duraction_min(self):
        return self.duration






In [3]:
file_timer = Timer("file load time")
df = spark.read.csv(SparkFiles.get(DATA_FILE), 
                    header=True, 
                    inferSchema= True)
df.collect()
file_timer.stop()

file load time duration: 0 minutes


In [4]:
def build_ngrams(inputCol, min_df, n=3):
    log.info(f'Creating TFIDF using min_df: {min_df}')
    
    tokenizer = [Tokenizer(inputCol = inputCol, outputCol = "words")]
    
    ngrams = [
        NGram(n=i, inputCol="words", outputCol="{0}_grams".format(i))
        for i in range(1, n + 1)
    ]

    vectorizers = [
        CountVectorizer(minDF=min_df, inputCol="{0}_grams".format(i),
            outputCol="{0}_counts".format(i))
        for i in range(1, n + 1)
    ]

    assembler = [VectorAssembler(
        inputCols=["{0}_counts".format(i) for i in range(1, n + 1)],
        outputCol="raw_features"
    )]
    
    idf = [IDF().setInputCol("raw_features").setOutputCol("features")]
#     idf = [IDF(minDocFreq=min_df).setInputCol("raw_features").setOutputCol("features")]

    return Pipeline(stages=tokenizer + ngrams + vectorizers + assembler + idf)



pipeline_timer = Timer("pipeline time")
# calculate a reasonable min_df
min_df = max(int(df.count() * DF_PERCENTAGE), 1)

pipeline = build_ngrams(FEATURE_COLUMN, min_df)
df = pipeline.fit(df).transform(df)
pipeline_timer.stop()


INFO:__main__:Creating TFIDF using min_df: 99


pipeline time duration: 0 minutes


In [5]:
df.printSchema()
pyu.show_df(df, ["star_rating", "features"], truncate=False)

root
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 1_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 2_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 3_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 1_counts: vector (nullable = true)
 |-- 2_counts: vector (nullable = true)
 |-- 3_counts: vector (nullable = true)
 |-- raw_features: vector (nullable = true)
 |-- features: vector (nullable = true)

+-----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Split training and test data

In [6]:
split_timer = Timer("train test split")
train, test = df.randomSplit([0.9, 0.1], seed=1)
split_timer.stop()


train_size = train.count()
test_size = test.count()

print(f'Training size: {train_size} Test size: {test_size}')

train test split duration: 0 minutes
Training size: 89544 Test size: 10023


# get number of features

In [7]:
type(train.select("features").limit(1).toPandas().features.values[0])
train.select("features").limit(1).toPandas().features.values[0].size

4159

# Assign class weights to handle imbalanced classes

In [8]:


# only do this for small files - takes too long for large datasets - we will custom compute this
# if DEBUG:
cw_timer = Timer("skelarn class weight")
labels = train.select("star_rating").toPandas().astype({"star_rating": np.int8})
class_weights = compute_class_weight('balanced', 
                                                  np.arange(1, N_CLASSES+1), 
                                                  labels.star_rating.tolist())
print(f'sklearn class weights: {class_weights}')
cw_timer.stop()
    


sklearn class weights: [1.43534503 3.06342798 2.17815617 1.1864847  0.37383989]
skelarn class weight duration: 0 minutes


In [9]:

train = train.withColumn("class_weights", lit(0))
train = train.withColumn("class_weights", when(train.star_rating == 1, class_weights[0]).otherwise(train.class_weights))
train = train.withColumn("class_weights", when(train.star_rating == 2, class_weights[1]).otherwise(train.class_weights))
train = train.withColumn("class_weights", when(train.star_rating == 3, class_weights[2]).otherwise(train.class_weights))
train = train.withColumn("class_weights", when(train.star_rating == 4, class_weights[3]).otherwise(train.class_weights))
train = train.withColumn("class_weights", when(train.star_rating == 5, class_weights[4]).otherwise(train.class_weights))


pyu.show_df(train, ["star_rating", "class_weights", "features"], 20, sample=True, truncate=True)


INFO:util.pyspark_util:sampling percentage: 0.00022335388189046726


+-----------+------------------+--------------------+
|star_rating|     class_weights|            features|
+-----------+------------------+--------------------+
|          1|  1.43534503486415|(4159,[4,6,17,21,...|
|          4|1.1864846959056579|(4159,[20,40,48,1...|
|          4|1.1864846959056579|(4159,[7,18,38,81...|
|          5|0.3738398914518318|(4159,[5,47,52,73...|
|          5|0.3738398914518318|(4159,[20,2476,40...|
|          5|0.3738398914518318|(4159,[0,1,7,8,11...|
|          1|  1.43534503486415|(4159,[0,11,23,45...|
|          4|1.1864846959056579|(4159,[8,31,77,90...|
|          4|1.1864846959056579|(4159,[54,55,64,2...|
|          5|0.3738398914518318|(4159,[20,37,268,...|
|          5|0.3738398914518318|(4159,[2,3,29,52,...|
|          5|0.3738398914518318|(4159,[9],[1.8649...|
|          5|0.3738398914518318|(4159,[9],[1.8649...|
|          5|0.3738398914518318|(4159,[0,2,6,7,9,...|
|          5|0.3738398914518318|(4159,[61,79,92,1...|
|          5|0.3738398914518

# Train our Model

In [10]:


import importlib
import util.constants as c
importlib.reload(pyu)
importlib.reload(mw)
importlib.reload(c)

feature_count = train.select("features").limit(1).toPandas().features.values[0].size

train_timer = Timer("traing time")
lr = LogisticRegression(labelCol="star_rating", 
                        featuresCol="features", 
                        weightCol="class_weights",
                        maxIter=100)


model = pyu.PysparkModel(model = lr,
                    train_df = train,
                    test_df = test,
                    label_column = "star_rating",
                    feature_column = "features",
                    n_classes = 5,
                         pipeline = pipeline,
                         file = DATA_FILE,
                         description=f'review_body-tfidf-df_none-ngram13-{df.count()}-{feature_count}-nolda-sampling_none{TEST_STR}',
                        model_dir="../../models")

report_dict, predict_test = model.run()
train_timer.stop()

pyu.show_df(predict_test, ["star_rating", "prediction", "rawPrediction", "probability"])



INFO:util.model_wrapper:derived name: LogisticRegression
INFO:util.model_wrapper:########################################
INFO:util.model_wrapper:Running model: name: LogisticRegression
	with file: /home/jupyter/dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-100k-preprocessed.csv
	with description: review_body-tfidf-df_none-ngram13-99567-4159-nolda-sampling_none-LogisticRegression-star_rating
	status: new
INFO:util.model_wrapper:########################################
INFO:util.time_util:Start timer for: train_time_min
INFO:util.time_util:End timer for: train_time_min
INFO:util.time_util:Total time for train_time_min: 0.32
INFO:util.time_util:Start timer for: model_save_time_min
INFO:util.pyspark_util:Saving model to file: ../../models/review_body-tfidf-df_none-ngram13-99567-4159-nolda-sampling_none-LogisticRegression-star_rating.pyspark
INFO:util.pyspark_util:Saving pipeline to file: ../../models/review_body-tfidf-df_none-ngram13-99567-4159-nolda-sampling_none-LogisticRegres

root
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 1_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 2_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 3_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 1_counts: vector (nullable = true)
 |-- 2_counts: vector (nullable = true)
 |-- 3_counts: vector (nullable = true)
 |-- raw_features: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



INFO:util.time_util:End timer for: cr_time_min
INFO:util.time_util:Total time for cr_time_min: 0.41
INFO:util.model_wrapper:calculating confusion matrix...
INFO:util.time_util:Start timer for: cm_time_min
INFO:util.time_util:End timer for: cm_time_min
INFO:util.time_util:Total time for cm_time_min: 0.37


traing time duration: 1 minutes
+-----------+----------+--------------------+--------------------+
|star_rating|prediction|       rawPrediction|         probability|
+-----------+----------+--------------------+--------------------+
|          1|       2.0|[-8.1948791419873...|[1.14526774859801...|
|          1|       1.0|[-8.1860780942214...|[3.76223062941299...|
|          1|       1.0|[-8.2341209709348...|[4.44985968567783...|
|          1|       1.0|[-8.1863929047636...|[5.44609080299316...|
|          1|       2.0|[-8.1943006037097...|[3.35845441805422...|
|          1|       1.0|[-8.1813635687497...|[2.05734359570510...|
|          1|       1.0|[-8.1803152040421...|[3.98463868179808...|
|          1|       1.0|[-8.2019638612832...|[2.90404724242306...|
|          1|       2.0|[-8.1879056376964...|[5.63631355110019...|
|          1|       2.0|[-8.1947269795437...|[6.37813046297268...|
+-----------+----------+--------------------+--------------------+
only showing top 10 rows



In [11]:
predict_test.printSchema()

root
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 1_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 2_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 3_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 1_counts: vector (nullable = true)
 |-- 2_counts: vector (nullable = true)
 |-- 3_counts: vector (nullable = true)
 |-- raw_features: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



# Evaluate our Model

Reference:
* https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#multiclass-classification

In [12]:
report_dict

{'model_name': 'LogisticRegression',
 'description': 'review_body-tfidf-df_none-ngram13-99567-4159-nolda-sampling_none-LogisticRegression-star_rating',
 'library': 'pyspark',
 'file': '/home/jupyter/dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-100k-preprocessed.csv',
 'model_file': '../../models/review_body-tfidf-df_none-ngram13-99567-4159-nolda-sampling_none-LogisticRegression-star_rating.pyspark',
 'pipeline_file': '../../models/review_body-tfidf-df_none-ngram13-99567-4159-nolda-sampling_none-LogisticRegression-star_rating.pipeline',
 'status': 'success',
 'status_date': '2019-11-22 18:26:20',
 'classification_report': '{"1": {"precision": 0.6406469760900141, "recall": 0.6483985765124555, "f1-score": 0.644499469402193, "support": 1405}, "2": {"precision": 0.23336968375136313, "recall": 0.31892697466467956, "f1-score": 0.26952141057934503, "support": 671}, "3": {"precision": 0.26875515251442705, "recall": 0.3622222222222222, "f1-score": 0.3085660198769522, "support": 900}, 

In [13]:
pprint(json.loads(report_dict["classification_report"]))

{'1': {'f1-score': 0.644499469402193,
       'precision': 0.6406469760900141,
       'recall': 0.6483985765124555,
       'support': 1405},
 '2': {'f1-score': 0.26952141057934503,
       'precision': 0.23336968375136313,
       'recall': 0.31892697466467956,
       'support': 671},
 '3': {'f1-score': 0.3085660198769522,
       'precision': 0.26875515251442705,
       'recall': 0.3622222222222222,
       'support': 900},
 '4': {'f1-score': 0.3812316715542522,
       'precision': 0.34980430528375733,
       'recall': 0.4188635032220269,
       'support': 1707},
 '5': {'f1-score': 0.77219207535579,
       'precision': 0.8518183871696409,
       'recall': 0.7061797752808989,
       'support': 5340},
 'accuracy': 0.5923376234660281,
 'macro avg': {'f1-score': 0.4752021293537065,
               'precision': 0.4688789009618405,
               'recall': 0.49091821038045663,
               'support': 10023},
 'weighted avg': {'f1-score': 0.6124262381164428,
                  'precision': 0.6429

In [14]:
print(np.array(json.loads(report_dict["confusion_matrix"])))

[[ 911  280  132   46   36]
 [ 221  214  150   61   25]
 [ 115  185  326  195   79]
 [  57  112  307  715  516]
 [ 118  126  298 1027 3771]]


In [15]:
import json

def calculate_score(cr: dict):
    
    values = []
    values.append(cr["1"]["recall"])
    values.append(cr["2"]["recall"])
    values.append(cr["3"]["recall"])
    values.append(cr["4"]["recall"])
    values.append(cr["5"]["precision"])
    
    mean = 0
    for v in values:
        if v == 0:
            mean = 0
            break
        else:
            mean += 1 / v
    if mean > 0:
        mean = len(values) / mean

    return mean




score = calculate_score(json.loads(report_dict['classification_report']))
print(f'Overall score: {score}')

Overall score: 0.45455028038919754


# Save Report

In [16]:
if os.path.exists(REPORT_FILE):
    report_df = pd.read_csv(REPORT_FILE, quotechar="'")
else:
    report_df = pd.DataFrame()
report_df = report_df.append(report_dict, ignore_index=True)
report_df.to_csv(REPORT_FILE, index=False, quotechar="'")
report_df

,classification_report,cm_time_min,confusion_matrix,cr_time_min,description,file,library,model_file,model_name,model_save_time_min,pipeline_file,predict_time_min,status,status_date,test_examples,test_features,total_time_min,train_examples,train_features,train_time_min
0,152,526.00,1885]]',0.35,review_body-tfidf-df_none-ngram13-49784-4254-n...,/home/jupyter/dataset/amazon_reviews/amazon_re...,pyspark,../../models/review_body-tfidf-df_none-ngram13...,LogisticRegression,0.06,../../models/review_body-tfidf-df_none-ngram13...,0.0,success,2019-11-22 07:50:22,5058.0,4254.0,1.00,44726.0,4254.0,0.30
1,"{""1"": {""precision"": 0.6406469760900141, ""recal...",0.32,"[[911, 280, 132, 46, 36], [221, 214, 150, 61, ...",0.45,review_body-tfidf-df_none-ngram13-99567-4159-n...,/home/jupyter/dataset/amazon_reviews/amazon_re...,pyspark,../../models/review_body-tfidf-df_none-ngram13...,LogisticRegression,0.05,../../models/review_body-tfidf-df_none-ngram13...,0.0,success,2019-11-22 07:53:26,10023.0,4159.0,1.15,89544.0,4159.0,0.33
2,"{""1"": {""precision"": 0.6127167630057804, ""recal...",0.28,"[[424, 173, 80, 31, 27], [90, 97, 73, 35, 19],...",0.30,review_body-tfidf-df_none-ngram13-49784-4254-n...,/home/jupyter/dataset/amazon_reviews/amazon_re...,pyspark,../../models/review_body-tfidf-df_none-ngram13...,LogisticRegression,0.04,../../models/review_body-tfidf-df_none-ngram13...,0.0,success,2019-11-22 18:23:18,5058.0,4254.0,0.89,44726.0,4254.0,0.27
3,"{""1"": {""precision"": 0.6406469760900141, ""recal...",0.37,"[[911, 280, 132, 46, 36], [221, 214, 150, 61, ...",0.41,review_body-tfidf-df_none-ngram13-99567-4159-n...,/home/jupyter/dataset/amazon_reviews/amazon_re...,pyspark,../../models/review_body-tfidf-df_none-ngram13...,LogisticRegression,0.04,../../models/review_body-tfidf-df_none-ngram13...,0.0,success,2019-11-22 18:26:20,10023.0,4159.0,1.14,89544.0,4159.0,0.32
